# Import Package

In [69]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.metrics import r2_score

# from sklearn.externals import joblib
import joblib

import pickle
import uuid



In [70]:
# Importing necessary libraries
from keras.models import Sequential
from keras.layers import Dense
from keras import callbacks
import tensorflow as tf


# Data

## Download Data

In [71]:
import os.path
if not os.path.isfile('Covid_Data.zip'):
  !gdown --id 1oPQDuqWnH9v72qJiiyqI5LXhdV0yGaPE
  !unzip 'Covid_Data.zip'


In [72]:
descriptor = ("d", "r", "c")
filenames = [r'time_series_19-covid-Deaths.csv',
             r'time_series_19-covid-Recovered.csv',
             r'time_series_19-covid-Confirmed.csv']

print(zip(descriptor, filenames))



## Load Data & Feature Enginering;dgk

In [73]:
dfs = []
DATAs = []
for descriptor_, fname in zip(descriptor, filenames):
    print("Working on:",'Input Data/'+ fname)
    df = pd.read_csv('Input Data/'+ fname)
    df.drop(columns=["Province/State","Country/Region"], inplace=True)
    DATA = np.array((0,0,0,0))

    
    for i, j in df.iterrows():
        # print(i, j)
        latitude = j['Lat']
        longitude = j['Long']
        
        for k,l in j.iteritems():
            if k=='Lat':
                continue
            if k=='Long':
                continue
            date = datetime.strptime(k, '%m/%d/%y')
            day = date - date.strptime("01/22/20", '%m/%d/%y')
            days = day.days
            #print(days)
            temp = np.array([j["Lat"], j['Long'], days, l])
            #print(temp)
            DATA = np.vstack((DATA,temp))  
            
    DATA = np.delete(DATA, 0,0)
    np.random.shuffle(DATA)
    
    dfs.append(df)
    DATAs.append(DATA)


Working on: Input Data/time_series_19-covid-Deaths.csv
Working on: Input Data/time_series_19-covid-Recovered.csv
Working on: Input Data/time_series_19-covid-Confirmed.csv


In [74]:
dfs[0]

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20
0,15.0000,101.0000,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,36.0000,138.0000,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,10,10,15,16
2,1.2833,103.8333,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28.1667,84.2500,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.5000,112.5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,42.5922,-83.3362,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,42.2791,-83.3362,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
403,39.5393,-75.6674,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,22.0000,-80.0000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
DATAs

[array([[ -30.5595,   22.9375,   22.    ,    0.    ],
        [  40.745 , -123.8695,    7.    ,    0.    ],
        [  43.9424,   12.4578,   35.    ,    0.    ],
        ...,
        [  27.8764,  -82.7779,   28.    ,    0.    ],
        [  42.5922,  -83.3362,   21.    ,    0.    ],
        [  47.5162,   14.5501,    4.    ,    0.    ]]),
 array([[  47.6477, -122.6413,    0.    ,    0.    ],
        [  39.5393,  -75.6674,   18.    ,    0.    ],
        [  12.2383,   -1.5616,   29.    ,    0.    ],
        ...,
        [  33.1795,  -96.493 ,   30.    ,    0.    ],
        [  30.3883,  -95.6963,   22.    ,    0.    ],
        [  21.    ,   78.    ,   14.    ,    0.    ]]),
 array([[  56.2639,    9.5018,    8.    ,    0.    ],
        [  34.3705, -119.1391,   18.    ,    0.    ],
        [  39.3999,   -8.2245,   30.    ,    0.    ],
        ...,
        [  42.2791,  -83.3362,   43.    ,    0.    ],
        [  38.0834, -122.7633,   25.    ,    0.    ],
        [  42.632 ,  -70.7829,   35.   

In [76]:
DATAs[0]

array([[ -30.5595,   22.9375,   22.    ,    0.    ],
       [  40.745 , -123.8695,    7.    ,    0.    ],
       [  43.9424,   12.4578,   35.    ,    0.    ],
       ...,
       [  27.8764,  -82.7779,   28.    ,    0.    ],
       [  42.5922,  -83.3362,   21.    ,    0.    ],
       [  47.5162,   14.5501,    4.    ,    0.    ]])

# Models

## Confirmed

In [77]:
DATA = DATAs[2]
input_data = DATA[:,:-1]
print(input_data.shape)
output_data = DATA[:, -1]
print(output_data.shape)
input_train, input_test, output_train, output_test = train_test_split(input_data, output_data)

(20706, 3)
(20706,)


### keras

In [78]:
X_train, X_test, y_train, y_test = input_train, input_test, output_train, output_test

In [79]:
model = Sequential()

model.add(Dense(4, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='relu'))


In [80]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,          # minimium amount of change to be considered an improvement
    patience=20,              # number of epochs to wait before deciding there is no significant improvement in training
    restore_best_weights=True,
)

In [81]:
model.compile(
    loss='mean_squared_error', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )

In [82]:
history = model.fit(X_train, y_train, 
                    batch_size = 1024, 
                    epochs = 100, 
                    validation_split=0.2,
                    callbacks=[early_stopping]
                    )

Epoch 1/100000
13/13 [==============================] - 2s 16ms/step - loss: 6481283.0000 - val_loss: 5991953.5000
Epoch 2/100000
13/13 [==============================] - 0s 7ms/step - loss: 6436310.0000 - val_loss: 6030565.0000
Epoch 3/100000
13/13 [==============================] - 0s 7ms/step - loss: 6445223.5000 - val_loss: 5989728.0000
Epoch 4/100000
13/13 [==============================] - 0s 6ms/step - loss: 6428353.5000 - val_loss: 5969086.0000
Epoch 5/100000
13/13 [==============================] - 0s 7ms/step - loss: 6344712.0000 - val_loss: 5918072.0000
Epoch 6/100000
13/13 [==============================] - 0s 8ms/step - loss: 6310404.5000 - val_loss: 6095036.0000
Epoch 7/100000
13/13 [==============================] - 0s 6ms/step - loss: 6549289.5000 - val_loss: 6094577.0000
Epoch 8/100000
13/13 [==============================] - 0s 6ms/step - loss: 6548386.5000 - val_loss: 6093505.5000
Epoch 9/100000
13/13 [==============================] - 0s 6ms/step - loss: 6547053.500

In [83]:
predictions = model.predict(X_test)
print('r2-score is', r2_score(y_pred=predictions, y_true=y_test))

162/162 [==============================] - 0s 2ms/step
r2-score is 0.0351292976520039


In [84]:
predictions

array([[   3.7981806],
       [   3.7981806],
       [   3.7981806],
       ...,
       [   3.7981806],
       [   3.7981806],
       [2605.4165   ]], dtype=float32)

### sk-learn

In [85]:
clf = MLPRegressor(hidden_layer_sizes = (4,4,4,4),
                   activation='relu',
                   solver='lbfgs',
                   learning_rate='constant',
                   learning_rate_init=0.1,
                   alpha=0.0001,
                   max_iter=10000000)
clf.fit(input_train, output_train)

MLPRegressor(hidden_layer_sizes=(4, 4, 4, 4), learning_rate_init=0.1,
             max_iter=10000000, solver='lbfgs')

In [86]:
clf.score(input_train, output_train)


1.1102230246251565e-16

In [87]:
clf.score(input_test, output_test)

-0.00011936719774263693

## Recovered

In [88]:
DATA = DATAs[1]
input_data = DATA[:,:-1]
print(input_data.shape)
output_data = DATA[:, -1]
print(output_data.shape)
input_train, input_test, output_train, output_test = train_test_split(input_data, output_data)

(20706, 3)
(20706,)


### keras

In [89]:
X_train, X_test, y_train, y_test = input_train, input_test, output_train, output_test

In [90]:
model = Sequential()

model.add(Dense(4, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='relu'))


In [91]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,          # minimium amount of change to be considered an improvement
    patience=20,              # number of epochs to wait before deciding there is no significant improvement in training
    restore_best_weights=True,
)

In [92]:
model.compile(
    loss='mean_squared_error', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )

In [93]:
history = model.fit(X_train, y_train, 
                    batch_size = 1024, 
                    epochs = 100000, 
                    validation_split=0.2,
                    callbacks=[early_stopping]
                    )

Epoch 1/100000
13/13 [==============================] - 2s 17ms/step - loss: 1325359.5000 - val_loss: 290346.1875
Epoch 2/100000
13/13 [==============================] - 0s 8ms/step - loss: 1325359.5000 - val_loss: 290346.1875
Epoch 3/100000
13/13 [==============================] - 0s 9ms/step - loss: 1325359.5000 - val_loss: 290346.1875
Epoch 4/100000
13/13 [==============================] - 0s 10ms/step - loss: 1325359.5000 - val_loss: 290346.1875
Epoch 5/100000
13/13 [==============================] - 0s 9ms/step - loss: 1325359.3750 - val_loss: 290346.1875
Epoch 6/100000
13/13 [==============================] - 0s 9ms/step - loss: 1325359.3750 - val_loss: 290346.1875
Epoch 7/100000
13/13 [==============================] - 0s 9ms/step - loss: 1325359.6250 - val_loss: 290346.1875
Epoch 8/100000
13/13 [==============================] - 0s 9ms/step - loss: 1325359.6250 - val_loss: 290346.1875
Epoch 9/100000
13/13 [==============================] - 0s 9ms/step - loss: 1325359.6250 - val

In [94]:
predictions = model.predict(X_test)
print('r2-score is', r2_score(y_pred=predictions, y_true=y_test))

162/162 [==============================] - 0s 1ms/step
r2-score is -0.0021060901117526676


In [95]:
predictions

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

### sk-learn

In [96]:
clf = MLPRegressor(hidden_layer_sizes = (4,4,4,4),
                   activation='relu',
                   solver='lbfgs',
                   learning_rate='constant',
                   learning_rate_init=0.1,
                   alpha=0.0001,
                   max_iter=10000000)
clf.fit(input_train, output_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPRegressor(hidden_layer_sizes=(4, 4, 4, 4), learning_rate_init=0.1,
             max_iter=10000000, solver='lbfgs')

In [97]:
clf.score(input_train, output_train)


0.5377848567152564

In [98]:
clf.score(input_test, output_test)

0.5301919841778722


## Dead

In [99]:
DATA = DATAs[0]
input_data = DATA[:,:-1]
print(input_data.shape)
output_data = DATA[:, -1]
print(output_data.shape)
input_train, input_test, output_train, output_test = train_test_split(input_data, output_data)

(20706, 3)
(20706,)


### keras

In [100]:
X_train, X_test, y_train, y_test = input_train, input_test, output_train, output_test

In [101]:
model = Sequential()

model.add(Dense(4, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='relu'))


In [102]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,          # minimium amount of change to be considered an improvement
    patience=20,              # number of epochs to wait before deciding there is no significant improvement in training
    restore_best_weights=True,
)

In [103]:
model.compile(
    loss='mean_squared_error', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )

In [104]:
history = model.fit(X_train, y_train, 
                    batch_size = 1024, 
                    epochs = 100000, 
                    validation_split=0.2,
                    callbacks=[early_stopping]
                    )

Epoch 1/100000
13/13 [==============================] - 2s 15ms/step - loss: 11650.3926 - val_loss: 6568.0425
Epoch 2/100000
13/13 [==============================] - 0s 8ms/step - loss: 11352.6914 - val_loss: 6568.0425
Epoch 3/100000
13/13 [==============================] - 0s 6ms/step - loss: 11352.6895 - val_loss: 6568.0425
Epoch 4/100000
13/13 [==============================] - 0s 5ms/step - loss: 11352.6924 - val_loss: 6568.0425
Epoch 5/100000
13/13 [==============================] - 0s 6ms/step - loss: 11352.6924 - val_loss: 6568.0425
Epoch 6/100000
13/13 [==============================] - 0s 6ms/step - loss: 11352.6914 - val_loss: 6568.0425
Epoch 7/100000
13/13 [==============================] - 0s 5ms/step - loss: 11352.6914 - val_loss: 6568.0425
Epoch 8/100000
13/13 [==============================] - 0s 6ms/step - loss: 11352.6914 - val_loss: 6568.0425
Epoch 9/100000
13/13 [==============================] - 0s 5ms/step - loss: 11352.6895 - val_loss: 6568.0425
Epoch 10/100000
13

In [105]:
predictions = model.predict(X_test)
print('r2-score is', r2_score(y_pred=predictions, y_true=y_test))

162/162 [==============================] - 1s 3ms/step
r2-score is -0.0020345375408667987


In [106]:
predictions

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

### sk-learn

In [107]:
clf = MLPRegressor(hidden_layer_sizes = (4,4,4,4),
                   activation='relu',
                   solver='lbfgs',
                   learning_rate='constant',
                   learning_rate_init=0.1,
                   alpha=0.0001,
                   max_iter=10000000)
clf.fit(input_train, output_train)

MLPRegressor(hidden_layer_sizes=(4, 4, 4, 4), learning_rate_init=0.1,
             max_iter=10000000, solver='lbfgs')

In [108]:
clf.score(input_train, output_train)


0.8086752877030592

In [109]:
clf.score(input_test, output_test)

0.7738306300808557